In [1]:
## Script EAPV completo - TRATAMENTO DOS DADOS

import pandas as pd
import datetime
import numpy as np
import re
from unidecode import unidecode
import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
print("Inicio: " + datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

Inicio: 07/03/2022 11:51:27


In [2]:
## PUXANDO TODOS OS ARQUIVOS NECESSÁRIOS

# Função remover acentos
def strip_accents(text):
    return str(unidecode(text))

## Banco E-SUS
df = pd.read_csv('Arquivos/eapv_08022022_09h23.csv', sep=';', encoding='utf8')
print(f'Banco importado sem tratamento: {df.shape}')

# Selecionar somente data da notificação até 08-02-2022 - CONFORME BOLETIM
df['Data da Notificação'] = pd.to_datetime(df['Data da Notificação'], format='%d/%m/%Y')
df = df[df['Data da Notificação'] <= '2022-02-08']

print(f'Banco completo até 08-02-2022 com as canceladas: {df.shape[0]}')
banco_completo = df.shape[0]

# Exclusão das fichas canceladas pelo número da notificação
# Transforma valores da coluna em strings
df['Número da Notificação'] = df['Número da Notificação'].astype(str)

# Carrega dataframe canceladas
df_canceladas = pd.read_csv('Arquivos/eapv_canceladas_08022022.csv', sep=';', encoding='utf8')
df_canceladas['Número da Notificação'] = df_canceladas['Número da Notificação'].astype(str)
print(f'Banco canceladas: {df_canceladas.shape[0]}')
banco_canceladas = df_canceladas.shape[0]

# Pesquisa canceladas no df original e elimina
notif_canceladas = df_canceladas['Número da Notificação'].to_list()
df = df[~df['Número da Notificação'].str.contains('(?i)' + '|'.join(notif_canceladas), na=False)]
print(f'Banco sem as canceladas: {df.shape[0]}')
print(f'Teste subtração: banco completo - canceladas = {banco_completo} - {banco_canceladas} = {banco_completo - banco_canceladas}')

## Dados nome do municipio e CRS
data_mun_crs = pd.read_csv('Arquivos/data_mun_crs.csv', sep=';', encoding='cp1252')
data_mun_crs.columns = [x.upper() for x in list(data_mun_crs.columns.values)]
data_mun_crs['MUNICIPIO DE RESIDENCIA'] = data_mun_crs.apply(lambda e: strip_accents(e['MUNICIPIO DE RESIDENCIA']), axis=1)

## Dados SIVEP - confirmados COVID-19 - do painel BI SES 
data_conf_painel = pd.read_csv('Arquivos/data_conf_painel_08_02.csv', sep=';', encoding='cp1252')
data_conf_painel.columns = [x.upper() for x in list(data_conf_painel.columns.values)]
#data_conf_painel['DATA SINTOMAS'] = data_conf_painel['DATA SINTOMAS'].dt.date
data_conf_painel['DATA SINTOMAS'] = pd.to_datetime(data_conf_painel['DATA SINTOMAS'])
data_conf_painel = data_conf_painel[data_conf_painel['DATA SINTOMAS'] >= '2021-01-05']
data_conf_painel.drop_duplicates(subset=['NOME', 'CPF', 'DATA SINTOMAS'], keep='last', inplace=True)

Banco importado sem tratamento: (17153, 189)
Banco completo até 08-02-2022 com as canceladas: 17150
Banco canceladas: 772
Banco sem as canceladas: 16378
Teste subtração: banco completo - canceladas = 17150 - 772 = 16378


In [4]:
# Dados EAPV
data_eapv = df.copy()
data_eapv = data_eapv[['Número da Notificação', 'Data da Notificação', 'Estado de Residência', 'Idade Evento', 'Sexo', 'CPF', 'Município de Residência',
                       'Data de Nascimento', 'Nome Completo da Mãe', 'Nome Completo', 'É profissional de saúde?', 'Raça/Cor', 'Comunidade/Povo Tradicional',
                       'Relação imunobiológico ao evento adverso', 'Nome do Fabricante', 'Imunobiológico (vacina)', 'Código Imunobiológico',
                       'Lote', 'Dose', 'Data da aplicação', 'Descrição do caso ', 'Reação / evento adverso', 'Código Evento Adverso',
                       'Data de início', 'Tipo de Evento', 'Classificação de gravidade', 'Gravidade', 'Desfecho (evolução do caso)',
                       'Data Desfecho', 'Gestante no momento da vacinação?', 'Doenças (CID10)', 'Descricao detalhada do Evento Adverso',
                       'Houve atendimento médico?', 'Tipo de Atendimento', 'Observações complementares', 'CNES Estabelecimento de saúde',
                       'Nome do exame', 'Resultado do exame', 'Data da realização do exame', 'Diagnóstico (CID-10)', 'Medicamento em uso anterior ou durante a vacinação',
                       'Nome do Medicamento', 'Relação do medicamento ao evento adverso', 'Comentário sobre o uso', 'Encerramento Municipal - Grave?',
                       'Encerramento Municipal - Evento adverso', 'Encerramento Municipal - Tipo', 'Encerramento Municipal - Diagnóstico', 'Encerramento Municipal - Causalidade',
                       'Encerramento Municipal - Conduta', 'Encerramento Municipal - Informe a conduta', 'Encerramento Municipal - Comentários', 'Encerramento Municipal - CNES Estabelecimento',
                       'Encerramento Municipal - Telefone Responsável', 'Encerramento Municipal - CPF Responsável', 'Encerramento Municipal - Nome do profissional',
                       'Encerramento Municipal - E-mail Responsável', 'Encerramento Municipal - Ocupação do Profissional ', 'Encerramento Municipal - Data Encerramento',
                       'Encerramento Estadual - Grave?', 'Encerramento Estadual - Evento adverso', 'Encerramento Estadual - Tipo', 'Encerramento Estadual - Diagnóstico',
                       'Encerramento Estadual - Causalidade', 'Encerramento Estadual - Conduta', 'Encerramento Estadual - Informe a conduta', 'Encerramento Estadual - Comentários',
                       'Encerramento Estadual - CNES Estabelecimento', 'Encerramento Estadual - Telefone Responsável', 'Encerramento Estadual - CPF Responsável', 'Encerramento Estadual - Nome do profissional',
                       'Encerramento Estadual - E-mail Responsável', 'Encerramento Estadual - Ocupação do Profissional ', 'Encerramento Estadual - Data Encerramento',
                       'Encerramento Federal - Grave?', 'Encerramento Federal - Evento adverso', 'Encerramento Federal - Tipo', 'Encerramento Federal - Diagnóstico',
                       'Encerramento Federal - Causalidade', 'Encerramento Federal - Conduta', 'Encerramento Federal - Informe a conduta', 'Encerramento Federal - Comentários',
                       'Encerramento Federal - CNES Estabelecimento', 'Encerramento Federal - Telefone Responsável', 'Encerramento Federal - CPF Responsável',
                       'Encerramento Federal - Nome do profissional', 'Encerramento Federal - E-mail Responsável', 'Encerramento Federal - Ocupação do Profissional ',
                       'Encerramento Federal - Data Encerramento', 'Encerramento Cifavi - Grave?', 'Encerramento Cifavi - Evento adverso', 'Encerramento Cifavi - Tipo',
                       'Encerramento Cifavi - Diagnóstico', 'Encerramento Cifavi - Causalidade', 'Encerramento Cifavi - Conduta', 'Encerramento Cifavi - Informe a conduta',
                       'Encerramento Cifavi - Comentários', 'Encerramento Cifavi - CNES Estabelecimento', 'Encerramento Cifavi - Telefone Responsável', 'Encerramento Cifavi - CPF Responsável',
                       'Encerramento Cifavi - Nome do profissional', 'Encerramento Cifavi - E-mail Responsável', 'Encerramento Cifavi - Ocupação do Profissional ',
                       'Encerramento Cifavi - Data Encerramento']]

# Colunas em Maiúsculo, sem acentos e sem espaços nas bordas
data_eapv.columns = [strip_accents(x.upper()).strip() for x in list(data_eapv.columns.values)]

# Tratamento dos dados
data_eapv = data_eapv.drop_duplicates()
print(f'Banco após drop duplicates: {data_eapv.shape}')

imunobiologico = ['oxford', 'astrazeneca', 'fiocruz', 'covishield', 'pfizer', 'comirnaty', 
                  'butantan', 'sinovac', 'coronavac', 'covid', 'janssen']
data_eapv = data_eapv[data_eapv['IMUNOBIOLOGICO (VACINA)'].str.contains('(?i)' + '|'.join(imunobiologico), na = False)]
print(f'Banco após filtrar somente as vacinas covid: {data_eapv.shape}')

data_eapv['NOME COMPLETO'] = data_eapv.apply(lambda e: strip_accents(e['NOME COMPLETO'].upper().strip()), axis=1)
data_eapv['MUNICIPIO DE RESIDENCIA'] = data_eapv.apply(lambda e: strip_accents(e['MUNICIPIO DE RESIDENCIA']), axis=1)
data_eapv = data_eapv.merge(data_mun_crs, how='left')

data_eapv['CPF'] = data_eapv['CPF'].str.replace('.', '').str.replace('-', '')

Banco após drop duplicates: (16378, 104)
Banco após filtrar somente as vacinas covid: (16263, 104)


In [5]:
# Usa coluna IDADE EVENTO e aplica inconsistente se < 12 e > 107 nas faixas etárias
data_eapv['FAIXA ETARIA'] = 'Inconsistente'
# Pfizer começou dia 18/07/2021 para maiores de 12 anos
# Se vacina = Pfizer e IDADE EVENTO >= 12 e < 18 - faixa etária (12 - 17)
data_eapv.loc[(data_eapv['IDADE EVENTO'] >=  5) & (data_eapv['IDADE EVENTO'] <= 11), 'FAIXA ETARIA'] = '05-11'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 12) & (data_eapv['IDADE EVENTO'] <= 17), 'FAIXA ETARIA'] = '12-17'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 18) & (data_eapv['IDADE EVENTO'] <= 19), 'FAIXA ETARIA'] = '18-19'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 20) & (data_eapv['IDADE EVENTO'] <= 24), 'FAIXA ETARIA'] = '20-24'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 25) & (data_eapv['IDADE EVENTO'] <= 29), 'FAIXA ETARIA'] = '25-29'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 30) & (data_eapv['IDADE EVENTO'] <= 34), 'FAIXA ETARIA'] = '30-34'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 35) & (data_eapv['IDADE EVENTO'] <= 39), 'FAIXA ETARIA'] = '35-39'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 40) & (data_eapv['IDADE EVENTO'] <= 44), 'FAIXA ETARIA'] = '40-44'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 45) & (data_eapv['IDADE EVENTO'] <= 49), 'FAIXA ETARIA'] = '45-49'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 50) & (data_eapv['IDADE EVENTO'] <= 54), 'FAIXA ETARIA'] = '50-54'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 55) & (data_eapv['IDADE EVENTO'] <= 59), 'FAIXA ETARIA'] = '55-59'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 60) & (data_eapv['IDADE EVENTO'] <= 64), 'FAIXA ETARIA'] = '60-64'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 65) & (data_eapv['IDADE EVENTO'] <= 69), 'FAIXA ETARIA'] = '65-69'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 70) & (data_eapv['IDADE EVENTO'] <= 74), 'FAIXA ETARIA'] = '70-74'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 75) & (data_eapv['IDADE EVENTO'] <= 79), 'FAIXA ETARIA'] = '75-79'
data_eapv.loc[(data_eapv['IDADE EVENTO'] >= 80) & (data_eapv['IDADE EVENTO'] <= 107), 'FAIXA ETARIA'] = '80+'

print(f'Banco completo somente com vacinas COVID-19: {data_eapv.shape}')
df_eapv_ei_covid = data_eapv.copy()

Banco completo somente com vacinas COVID-19: (16263, 106)


In [6]:
## Relatório 1 - Erro de Imunizacao
df_erro_imunizacao = data_eapv.loc[data_eapv['TIPO DE EVENTO'].str.contains("(?i)erro de imunização")]
data_eapv = data_eapv.loc[~data_eapv['NUMERO DA NOTIFICACAO'].isin(df_erro_imunizacao['NUMERO DA NOTIFICACAO'])]
print(f'Banco Erros de Imunização: {df_erro_imunizacao.shape}')
print(f'Banco EAPV sem Erros de Imunização: {data_eapv.shape}')

# Cria um relatorio para conferirmos no excel as faixas etarias inconscistentes
df_inconsistentes = data_eapv[(data_eapv['FAIXA ETARIA'] == 'Inconsistente')]
df_inconsistentes.to_excel('faixa_etaria_inconsistente.xlsx', index=False)
print(f'Total de fichas com faixa etária inconsistente: {df_inconsistentes.shape[0]}')

#Excluir faixas etárias Inconsistentes 
data_eapv = data_eapv.loc[data_eapv['FAIXA ETARIA'] != 'Inconsistente']
print(f'Banco EAPV após excluir faixas etárias inconsistentes: {data_eapv.shape}')

# Normaliza as colunas com texto de campo aberto para remover acentos e deixar tudo minusculo
cols = ['DESCRICAO DETALHADA DO EVENTO ADVERSO', 'DESCRICAO DO CASO', 'OBSERVACOES COMPLEMENTARES', 'NOME DO EXAME', 'RESULTADO DO EXAME',
        'ENCERRAMENTO MUNICIPAL - EVENTO ADVERSO', 'ENCERRAMENTO ESTADUAL - EVENTO ADVERSO', 'ENCERRAMENTO FEDERAL - EVENTO ADVERSO', 
        'ENCERRAMENTO CIFAVI - EVENTO ADVERSO', 'GRAVIDADE', 'DESFECHO (EVOLUCAO DO CASO)']
data_eapv[cols] = data_eapv[cols].astype(str).apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower())
data_eapv['REACAO / EVENTO ADVERSO'] = data_eapv.apply(lambda x: strip_accents(x['REACAO / EVENTO ADVERSO']), axis=1)

Banco Erros de Imunização: (3415, 106)
Banco EAPV sem Erros de Imunização: (12848, 106)
Total de fichas com faixa etária inconsistente: 15
Banco EAPV após excluir faixas etárias inconsistentes: (12833, 106)


In [7]:
## Separar as vacinas (antes do relatorio de covid)

# 1 split da vacina E colunas de info da vacina como data de aplicação
# 2 simplificar nome das vacinas (coluna VACINA)
# 3 Merge com data_eapv
# 4 strip nos caracteres poluindo a data da aplicação
# 5 drop duplicates (ignorando a coluna IMUNOBIOLOGICO)

# 1 split da vacina E colunas de info da vacina como data de aplicação
df_vacinas_separadas = pd.DataFrame()

for index, row in data_eapv.iterrows():
    vacinas = row['IMUNOBIOLOGICO (VACINA)'].split('| ')
    data_aplicacao = row['DATA DA APLICACAO'].split('| ')
        
    i = 0
    dados = {}
    for vacina in vacinas:
        # Trata os dados e concatena depois no dataframe
        dados.setdefault("NUMERO DA NOTIFICACAO", []).append(row['NUMERO DA NOTIFICACAO'])
        dados.setdefault("IMUNOBIOLOGICO (VACINA)", []).append(vacina)
        dados.setdefault("DATA DA APLICACAO", []).append(data_aplicacao[i])

        i += 1

    df_vacinas_separadas = pd.concat([df_vacinas_separadas, pd.DataFrame(dados)])
df_vacinas_separadas

# 2 simplificar nome das vacinas (coluna VACINA) e excluir outras vacinas não covid
df_vacinas_separadas['VACINA'] = pd.Series()
df_vacinas_separadas['IMUNOBIOLOGICO (VACINA)'] = df_vacinas_separadas['IMUNOBIOLOGICO (VACINA)'].str.lower()

pfizer = ['pfizer', 'comirnaty']
astra = ['astrazeneca', 'covishield', 'oxford', 'fiocruz']
butantan = ['butantan', 'coronavac', 'sinovac']

df_vacinas_separadas.loc[df_vacinas_separadas['IMUNOBIOLOGICO (VACINA)'].str.contains('janssen'), 'VACINA'] = 'Janssen'
df_vacinas_separadas.loc[df_vacinas_separadas['IMUNOBIOLOGICO (VACINA)'].str.contains('(?i)' + '|'.join(pfizer)), 'VACINA'] = 'Pfizer'
df_vacinas_separadas.loc[df_vacinas_separadas['IMUNOBIOLOGICO (VACINA)'].str.contains('(?i)' + '|'.join(astra)), 'VACINA'] = 'Astrazeneca'
df_vacinas_separadas.loc[df_vacinas_separadas['IMUNOBIOLOGICO (VACINA)'].str.contains('(?i)' + '|'.join(butantan)), 'VACINA'] = 'Butantan'

# Exclui linhas com VACINA = nan pois são outras vacinas não covid
df_vacinas_separadas = df_vacinas_separadas[df_vacinas_separadas['VACINA'].notna()]


# 3 MERGE com data_eapv para atualizar as novas linhas com as vacinas separadas
# Antes do merge excluir colunas que não interessam mais
df_vacinas_separadas.drop('IMUNOBIOLOGICO (VACINA)', axis=1, inplace=True)
data_eapv.drop(['IMUNOBIOLOGICO (VACINA)', 'DATA DA APLICACAO'], axis=1, inplace=True)

# Merge com dataframe original para adicionar as demais informações pelo numero da notificação
data_eapv = pd.merge(df_vacinas_separadas, data_eapv, how='left', on='NUMERO DA NOTIFICACAO')

# 4 strip nos caracteres poluindo a data da aplicação
import re
data_eapv['DATA DA APLICACAO'] = data_eapv['DATA DA APLICACAO'].str.replace('[0-9]: ', '')
data_eapv['DATA DA APLICACAO'] = data_eapv['DATA DA APLICACAO'].str.strip()


# 5 drop duplicates pois a VACINA tem duplicação de informações (possível ver após simplificar o nome da coluna IMUNOBIOLOGICO)
# Ver se tem que colocar uma condição para dropar porque as vezes tem a mesma vacina lançada com 1 dia de diferença
data_eapv = data_eapv.drop_duplicates(keep='first')
data_eapv = data_eapv.drop_duplicates(subset=['NUMERO DA NOTIFICACAO', 'DATA DA APLICACAO'])

In [8]:
######### ATENÇAO #############
# Pfizer inserir na ABT somente quando aplicação em data > 04-05-2021 # iniciou pfizer POA - 04/05  ->  pfizer RS - 24-05
# Para esta análise excluímos as Pfizer com data anterior a 04-05 e contamos como inconsistentes, pois foram notificações erradas
data_eapv = data_eapv[~((data_eapv['VACINA'] == 'Pfizer') & 
                        (data_eapv['DATA DA NOTIFICACAO'] <= '2021-05-04'))]

# Janssen inserir na ABT somente quando aplicação em data > 22-06-2021
# Para esta análise excluímos as Janssen com data anterior a 22-06 e contamos como inconsistentes, pois foram notificações erradas
data_eapv = data_eapv[~((data_eapv['VACINA'] == 'Janssen') & 
                        (data_eapv['DATA DA NOTIFICACAO'] <= '2021-06-22'))]

print(f'Banco EAPV sem vacinas dubias e Pfizer e Janssen com data errada: {data_eapv.shape}')

# TESTAR PRIMEIRO - Elimina as duplicatas por VACINA e DOSE, mantendo a última notificação no banco
#data_eapv.drop_duplicates(subset=['VACINA', 'DOSE'], keep='last', inplace=True)
#print(f'Banco EAPV após eliminar fichas com nome e dose duplicados: {data_eapv.shape}')

Banco EAPV sem vacinas dubias e Pfizer e Janssen com data errada: (14185, 106)


In [9]:
## Separamos as vacinas com a data da aplicação de cada uma,
# agora é preciso separar os eventos para ver qual evento se relaciona com qual vacina pela data
# Separar as Reações adversas por data que ocorreram
df_reacao = pd.DataFrame()
cont = 0

data_eapv['DATA DE INICIO'] = data_eapv['DATA DE INICIO'].fillna("")

for index, row in data_eapv.iterrows():
    rea_eve_adverso = row['REACAO / EVENTO ADVERSO'].split('| ')
    data_inicio = row['DATA DE INICIO'].split('| ')
    cod_eve_adverso = row['CODIGO EVENTO ADVERSO'].split('| ')
    gravidade = row['CLASSIFICACAO DE GRAVIDADE'].split('| ')
        
    i = 0
    dados = {}
    for rea in rea_eve_adverso:
        # Trata os dados e concatena depois no dataframe
        dados.setdefault("NUMERO DA NOTIFICACAO", []).append(row['NUMERO DA NOTIFICACAO'])
        dados.setdefault("REACAO / EVENTO ADVERSO", []).append(rea)
        dados.setdefault("CODIGO EVENTO ADVERSO", []).append(cod_eve_adverso[i])
        dados.setdefault("DATA DE INICIO", []).append(data_inicio[i])
        dados.setdefault("CLASSIFICACAO DE GRAVIDADE", []).append(gravidade[i])

        i += 1

    df_reacao = pd.concat([df_reacao, pd.DataFrame(dados)])

df_reacao

,NUMERO DA NOTIFICACAO,REACAO / EVENTO ADVERSO,CODIGO EVENTO ADVERSO,DATA DE INICIO,CLASSIFICACAO DE GRAVIDADE
0,1432100162526,1: Dor na garganta,1: 10033494,1: 22/10/2021,1: Não grave
1,1432100162526,2: Mialgia,2: 10028411,2: 22/10/2021,2: Não grave
2,1432100162526,3: Febre,3: 10016558,3: 22/10/2021,3: Não grave
0,1432100162541,1: Cefaleia,1: 10019211,1: 22/10/2021,1: Não grave
1,1432100162541,2: Nausea,2: 10028813,2: 22/10/2021,2: Não grave
...,...,...,...,...,...
0,1432100010677,1: Cefaleia,1: 10019211,1: 01/02/2021,1: Não grave
0,1432100010682,1: Dor localizada,1: 10050483,1: 03/02/2021,1: Não grave
1,1432100010682,2: Dor de cabeca,2: 10019198,2: 03/02/2021,2: Não grave
2,1432100010682,3: Dor atras dos olhos,3: 10033381,3: 03/02/2021,3: Não grave


In [10]:
# Fazer merge com o dataframe original para pegar o resto das colunas
# Primeiro excluir colunas que foram tratadas no df_reacao:
data_eapv.drop(['REACAO / EVENTO ADVERSO', 'CODIGO EVENTO ADVERSO', 'DATA DE INICIO', 'CLASSIFICACAO DE GRAVIDADE'], axis=1, inplace=True)

In [11]:
# Merge pelo numero da notificacao
data_eapv = data_eapv.merge(df_reacao, on='NUMERO DA NOTIFICACAO', how='left')

In [12]:
# Eliminar a numeração das strings
data_eapv['REACAO / EVENTO ADVERSO'] = data_eapv['REACAO / EVENTO ADVERSO'].str.replace('[0-9]*: ', '').str.strip()
data_eapv['CODIGO EVENTO ADVERSO'] = data_eapv['CODIGO EVENTO ADVERSO'].str.replace('[0-9]*: ', '').str.strip()
data_eapv['CLASSIFICACAO DE GRAVIDADE'] = data_eapv['CLASSIFICACAO DE GRAVIDADE'].str.replace('[0-9]*: ', '').str.strip()
data_eapv['DATA DE INICIO'] = data_eapv['DATA DE INICIO'].str.replace('[0-9]*: ', '').str.strip()

In [13]:
print(data_eapv.shape, df_reacao.shape)

(47864, 106) (39438, 5)


In [14]:
# Verificar se os eventos se relacionam com a vacina pela data da aplicação x data de início dos sintomas
# Transformar as colunas de data em datetime
data_eapv['DATA DA APLICACAO'] = pd.to_datetime(data_eapv['DATA DA APLICACAO'], format='%d/%m/%Y', errors='coerce')
data_eapv['DATA DE INICIO'] = pd.to_datetime(data_eapv['DATA DE INICIO'], format='%d/%m/%Y', errors='coerce')

# Problema - DATA DE INICIO em branco
# Os casos sem data de início dos sintomas tratar na mão no Excel (verificar na descrição do caso se consegue descobrir a data de inicio dos sintomas)
teste_excluidas = pd.concat([data_eapv[~(data_eapv['DATA DE INICIO'].between(data_eapv['DATA DA APLICACAO'], (data_eapv['DATA DA APLICACAO'] + datetime.timedelta(days=29))))]])
#teste_excluidas.to_excel('teste_excluidas.xlsx')

# Excluir os casos que a data dos sintomas não ocorreu em até 30 dias após a data da aplicação da vacina
data_eapv = pd.concat([data_eapv[data_eapv['DATA DE INICIO'].between(data_eapv['DATA DA APLICACAO'], (data_eapv['DATA DA APLICACAO'] + datetime.timedelta(days=29)))]])
data_eapv.shape

(38352, 106)

In [15]:
## Relatório 2 - EAPV com Infecção por COVID
# 1) PESQUISA DE COVID NO BANCO ESUS EAPV
#cria a coluna com o nome do exame simplificada
data_eapv['NOME_EXAME'] = pd.Series(dtype=str)
data_eapv.loc[data_eapv['NOME DO EXAME'].str.contains('pcr', na = False), 'NOME_EXAME'] = 'pcr'
data_eapv.loc[data_eapv['NOME DO EXAME'].str.contains('antigeno', na = False), 'NOME_EXAME'] = 'antigeno'
data_eapv.loc[data_eapv['NOME DO EXAME'].str.contains('swab', na = False), 'NOME_EXAME'] = 'covid'

#cria a coluna com o resultado do exame simplificada
data_eapv['RESULTADO_EXAME'] = pd.Series(dtype=str)
data_eapv['RESULTADO DO EXAME'] = data_eapv['RESULTADO DO EXAME'].str.replace('nao detectavel', 'negativo')\
                                                                 .str.replace('nao detectado', 'negativo')\
                                                                 .str.replace('nao reagente', 'negativo')
resultado_exame = ['positivo', 'detectavel', 'detectado', 'reagente']
data_eapv.loc[data_eapv['RESULTADO DO EXAME'].str.contains('|'.join(resultado_exame), na = False), 'RESULTADO_EXAME'] = 'positivo'

# trata colunas para datetime
data_eapv['DATA DA REALIZACAO DO EXAME'] = pd.to_datetime(data_eapv['DATA DA REALIZACAO DO EXAME'], errors='coerce')
data_eapv['DATA DE INICIO'] = pd.to_datetime(data_eapv['DATA DE INICIO'], errors='coerce')

#cria a coluna covid_positivo com nome_exame + resultado_exame ou outras condições
data_eapv['COVID_POSITIVO'] = pd.Series(dtype=str)
cols_ea = ['ENCERRAMENTO MUNICIPAL - EVENTO ADVERSO',  'ENCERRAMENTO ESTADUAL - EVENTO ADVERSO', 
           'ENCERRAMENTO FEDERAL - EVENTO ADVERSO', 'ENCERRAMENTO CIFAVI - EVENTO ADVERSO']
cols_diag = ['ENCERRAMENTO MUNICIPAL - DIAGNOSTICO', 'ENCERRAMENTO ESTADUAL - DIAGNOSTICO', 
             'ENCERRAMENTO FEDERAL - DIAGNOSTICO', 'ENCERRAMENTO CIFAVI - DIAGNOSTICO']
data_eapv[cols_ea + cols_diag] = data_eapv[cols_ea + cols_diag].astype(str)

data_eapv.loc[(((data_eapv['NOME_EXAME'].str.contains('(?i)pcr')) |
                (data_eapv['NOME_EXAME'].str.contains('(?i)antigeno')) |
                (data_eapv['NOME_EXAME'].str.contains('(?i)covid')))  &
                (data_eapv['RESULTADO_EXAME'].str.contains('(?i)positivo'))), 
              'COVID_POSITIVO'] = 'covid+'
              
              
data_eapv.loc[(data_eapv['ENCERRAMENTO MUNICIPAL - EVENTO ADVERSO'].str.contains('\\bCOVID-19\\b', na = False)) |
              (data_eapv['ENCERRAMENTO ESTADUAL - EVENTO ADVERSO'].str.contains('\\bCOVID-19\\b', na = False)) |
              (data_eapv['ENCERRAMENTO FEDERAL - EVENTO ADVERSO'].str.contains('\\bCOVID-19\\b', na = False)) |
              (data_eapv['ENCERRAMENTO CIFAVI - EVENTO ADVERSO'].str.contains('\\bCOVID-19\\b', na = False)) |
              (data_eapv['ENCERRAMENTO MUNICIPAL - DIAGNOSTICO'].str.contains('(?i)b342|u071|u072|b972|u04', na=False)) |
              (data_eapv['ENCERRAMENTO ESTADUAL - DIAGNOSTICO'].str.contains('(?i)b342|u071|u072|b972|u04', na=False)) |
              (data_eapv['ENCERRAMENTO FEDERAL - DIAGNOSTICO'].str.contains('(?i)b342|u071|u072|b972|u04', na=False)) |
              (data_eapv['ENCERRAMENTO CIFAVI - DIAGNOSTICO'].str.contains('(?i)b342|u071|u072|b972|u04', na=False)) |
              (data_eapv['DIAGNOSTICO (CID-10)'].str.contains('(?i)b342|u071|u072|b972|u04', na=False)) |
              (data_eapv['REACAO / EVENTO ADVERSO'].str.contains('(?i)teste de covid-19 por pcr positivo', na=False)), 'COVID_POSITIVO'] = 'covid+'

print(f'Banco EAPV antes de merge com SIVEP: {data_eapv.shape}')

df_covid = data_eapv[data_eapv['COVID_POSITIVO'] == 'covid+']
print(f'Banco COVID antes do merge com SIVEP: {df_covid.shape}')

data_eapv = data_eapv.loc[data_eapv['COVID_POSITIVO'] != 'covid+']
print(f'Banco EAPV sem  COVID antes do merge com SIVEP: {data_eapv.shape}')

Banco EAPV antes de merge com SIVEP: (38352, 109)
Banco COVID antes do merge com SIVEP: (2388, 109)
Banco EAPV sem  COVID antes do merge com SIVEP: (35964, 109)


In [16]:
# 2) PESQUISA DE COVID NO BANCO SIVEP (MERGE PELO CPF)

# FOI ALTERADO, POIS AGORA TEMOS A DATA DE APLICAÇÃO DE CADA UMA DAS VACINAS
# inserida a 'DATA DA REALIZACAO DO EXAME' na condição
# A partir de agora gerar o relatório de covid a partir da base evento adverso 
# (não tem como usar a base limpa pois nela as vacinas não estão separadas e não temos como bater a data da aplicação)
# verifica se a data da realização do exame de covid ocorreu no range de 14 dias antes ou 14 dias após a data de inicio dos sintomas

# Merge data_eapv com dados do painel (SIVEP+ESUS) confirmados COVID
df_eapv_sivep = pd.merge(data_eapv.loc[pd.notnull(data_eapv['CPF'])],
                          data_conf_painel.loc[pd.notnull(data_conf_painel['CPF'])],
                          on=  ['CPF'],
                          how = 'left')

# Excluir os casos que a data dos sintomas está fora da range 14 dias antes e 14 dias após a vacina
df_eapv_sivep = pd.concat([df_eapv_sivep[df_eapv_sivep['DATA SINTOMAS'].between((df_eapv_sivep['DATA DA APLICACAO'] - datetime.timedelta(days=14)),
                                                                                (df_eapv_sivep['DATA DA APLICACAO'] + datetime.timedelta(days=14)))]])

print(f'Banco SIVEP COVID+ : {df_eapv_sivep.shape}')

# Se as notificações do data_eapv estiverem entre as notificações do df_eapv_sivep, deletar
data_eapv = data_eapv.loc[~data_eapv['NUMERO DA NOTIFICACAO'].isin(df_eapv_sivep['NUMERO DA NOTIFICACAO'])]
print(f'Banco EAPV após exclusão dos covid+ do SIVEP: {data_eapv.shape}')

# Se as notificações do df_covid não estiverem entre as notificações do df_eapv_sivep, adicionar
df_covid = pd.concat([df_covid, df_eapv_sivep], join='outer')
df_covid['COVID_POSITIVO'] = 'covid+'
print(f'Banco COVID após inclusão dos covid+ do SIVEP: {df_covid.shape}')
print('Foram excluídas do Banco EAPV as fichas em que a data dos sintomas de COVID ocorreu no range de 14 dias antes '
      'e 14 dias após a aplicação da vacina. Se os sintomas ocorrem nesse período se atribuem a COVID e não à vacina.')

Banco SIVEP COVID+ : (984, 113)
Banco EAPV após exclusão dos covid+ do SIVEP: (34976, 109)
Banco COVID após inclusão dos covid+ do SIVEP: (3372, 113)
Foram excluídas do Banco EAPV as fichas em que a data dos sintomas de COVID ocorreu no range de 14 dias antes e 14 dias após a aplicação da vacina. Se os sintomas ocorrem nesse período se atribuem a COVID e não à vacina.


In [17]:
## Relatório 3 - Eventos adversos com óbitos
## Esta coluna nova com a informação do óbito tem que ser feita depois do split das reações, pois é preciso saber qual vacina
# está temporalmente associada com o óbito
data_eapv['OBITO'] = 'nao'
obito = ['obito', 'falec', '\\bmorte\\b'] #Pesquisa por toda palavra 'morte'

data_eapv.loc[(data_eapv['DESCRICAO DETALHADA DO EVENTO ADVERSO'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['DESCRICAO DO CASO'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['GRAVIDADE'].str.contains('ocasione o obito', na = False)), 'OBITO'] = 'provavel'

data_eapv.loc[(data_eapv['OBSERVACOES COMPLEMENTARES'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['ENCERRAMENTO MUNICIPAL - EVENTO ADVERSO'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['ENCERRAMENTO ESTADUAL - EVENTO ADVERSO'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['ENCERRAMENTO FEDERAL - EVENTO ADVERSO'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['ENCERRAMENTO CIFAVI - EVENTO ADVERSO'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['REACAO / EVENTO ADVERSO'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['DESFECHO (EVOLUCAO DO CASO)'].str.contains('(?i)' + '|'.join(obito), na = False)), 'OBITO'] = 'sim'


# Banco preliminar de óbitos (necessita conferência manual)
df_obitos = data_eapv[(data_eapv['OBITO'] == 'sim') | (data_eapv['OBITO'] == 'provavel')]

# O banco óbitos está por evento, para saber n de pessoas:
df_obitos['NOME COMPLETO'].nunique()

208

EXPORTAR RELATÓRIOS

In [21]:
# Relatório evento adverso com vacinas separadas, com óbitos e sem covid
df_erro_imunizacao.to_csv('Relatorios/erro_imunizacao.csv', sep=';', encoding='utf-8-sig', index=False)
df_covid.to_excel('Relatorios/infeccao_covid.xlsx', index=False)
data_eapv.to_csv('Relatorios/evento_adverso.csv', sep=';', encoding='utf-8-sig', index=False)
#data_eapv.to_excel(r'Relatorios/evento_adverso.xlsx', index=False)

print('ok')
print("Término: " + datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

ok
Término: 07/03/2022 12:00:56
